# HW2P2: Face Classification and Verification


Congrats on coming to the second homework in 11785: Introduction to Deep Learning. This homework significantly longer and tougher than the previous homework. You have 2 sub-parts as outlined below. Please start early! 


*   Face Recognition: You will be writing your own CNN model to tackle the problem of classification, consisting of 7000 identities
*   Face Verification: You use the model trained for classification to evaluate the quality of its feature embeddings, by comparing the similarity of known and unknown identities

For this HW, you only have to write code to implement your model architecture. Everything else has been provided for you, on the pretext that most of your time will be used up in developing the suitable model architecture for achieving satisfactory performance.

Common errors which you may face in this homeworks (because of the size of the model)


*   CUDA Out of Memory (OOM): You can tackle this problem by (1) Reducing the batch size (2) Calling `torch.cuda.empty_cache()` and `gc.collect()` (3) Finally restarting the runtime



How to run this notebook? You just run everything in sequence. I needed to run for more than 100 epochs and save it and continue to reach the accuracy level. 

# Preliminaries

In [ ]:
!nvidia-smi # to see what GPU you have

Thu Oct 27 04:56:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install wandb --quiet

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 385, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/comm

In [ ]:
import torch
from torchsummary import summary
import torchvision #This library is used for image-based operations (Augmentations)
import os
import gc
from tqdm import tqdm
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import glob
import wandb
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [ ]:
# import os.path as path 
# if not path.exists("/content/drive"):
#     print("meow")
#     !sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
#     !sudo apt-get update -qq 2>&1 > /dev/null
#     !sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
#     !google-drive-ocamlfuse

#     !sudo apt-get install -qq w3m # to act as web browser 
#     !xdg-settings set default-web-browser w3m.desktop # to set default browser
#     %cd /content
#     !mkdir drive
#     %cd drive
#     !mkdir MyDrive
#     %cd ..
#     %cd ..
#     !google-drive-ocamlfuse /content/drive/MyDrive

meow
W: GPG error: https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease: The following signatures couldn't be verified because the public key is not available: NO_PUBKEY A4B469963BF863CC
E: The repository 'https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease' is no longer signed.
W: GPG error: https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease: The following signatures couldn't be verified because the public key is not available: NO_PUBKEY A4B469963BF863CC
E: The repository 'https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease' is no longer signed.


debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
de

In [ ]:
from google.colab import drive # Link your drive if you are a colab user
drive.mount('/content/drive') # Models in this HW take a long time to get trained and make sure to save it her

NotImplementedError: ignored

# TODOs
As you go, please read the code and keep an eye out for TODOs!

# Download Data from Kaggle

In [ ]:
# TODO: Use the same Kaggle code from HW1P2
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"joonghochoi","key":"7420d9855539804c3a24a55d7add20dc"}') 
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

     |████████████████████████████████| 59 kB 3.6 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73275 sha256=61a6aad7348273e058980a34ba4b22b03ff92ef11e2e30fe4eebf26e0c4745a8
  Stored in directory: /root/.cache/pip/wheels/de/f7/d8/c3902cacb7e62cb611b1ad343d7cc07f42f7eb76ae3a52f3d1
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
!mkdir '/content/data'

!kaggle competitions download -c 11-785-f22-hw2p2-classification
!unzip -qo '11-785-f22-hw2p2-classification.zip' -d '/content/data'

!kaggle competitions download -c 11-785-f22-hw2p2-verification
!unzip -qo '11-785-f22-hw2p2-verification.zip' -d '/content/data'

100% 2.36G/2.37G [00:13<00:00, 184MB/s]
100% 2.37G/2.37G [00:13<00:00, 186MB/s]
 71% 12.0M/16.8M [00:00<00:00, 115MB/s]
100% 16.8M/16.8M [00:00<00:00, 145MB/s]


# Configs

In [ ]:
config = {
    'batch_size': 128, # Increase this if your GPU can handle it
    'lr': 0.0015,
    'epochs': 100, # 10 epochs is recommended ONLY for the early submission - you will have to train for much longer typically.
    # Include other parameters as needed.
    #after 10 epochs, 25%
}

# Classification Dataset

In [ ]:
DATA_DIR = '/content/data/11-785-f22-hw2p2-classification/'# TODO: Path where you have downloaded the data
TRAIN_DIR = os.path.join(DATA_DIR, "classification/train") 
VAL_DIR = os.path.join(DATA_DIR, "classification/dev")
TEST_DIR = os.path.join(DATA_DIR, "classification/test")

# Transforms using torchvision - Refer https://pytorch.org/vision/stable/transforms.html

#Shape of image:  torch.Size([3, 224, 224])
train_transforms = torchvision.transforms.Compose([ 
    # Implementing the right transforms/augmentation methods is key to improving performance.
                    torchvision.transforms.RandomHorizontalFlip(p=0.5),#horizontal flip
                    torchvision.transforms.ColorJitter(brightness=0.1,contrast=0.2,saturation=0.2),#color jitter   
                    #torchvision.transforms.RandomCrop(size=224),#random crop  

                    torchvision.transforms.RandAugment(num_ops=6), #changed from default 2 to 5
                    torchvision.transforms.RandomResizedCrop(size=224,scale=(0.25,1)),
                    torchvision.transforms.ToTensor() 

                    ])
# Most torchvision transforms are done on PIL images. So you convert it into a tensor at the end with ToTensor()
# But there are some transforms which are performed after ToTensor() : e.g - Normalization
# Normalization Tip - Do not blindly use normalization that is not suitable for this dataset

val_transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])


train_dataset = torchvision.datasets.ImageFolder(TRAIN_DIR, transform = train_transforms)
val_dataset = torchvision.datasets.ImageFolder(VAL_DIR, transform = val_transforms)
# You should NOT have data augmentation on the validation set. Why?


# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = config['batch_size'], 
                                           shuffle = True,num_workers = 4, pin_memory = True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = config['batch_size'], 
                                         shuffle = False, num_workers = 2)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# You can do this with ImageFolder as well, but it requires some tweaking
class ClassificationTestDataset(torch.utils.data.Dataset):

    def __init__(self, data_dir, transforms):
        self.data_dir   = data_dir
        self.transforms = transforms

        # This one-liner basically generates a sorted list of full paths to each image in the test directory
        self.img_paths  = list(map(lambda fname: os.path.join(self.data_dir, fname), sorted(os.listdir(self.data_dir))))

    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        return self.transforms(Image.open(self.img_paths[idx]))

In [ ]:
test_dataset = ClassificationTestDataset(TEST_DIR, transforms = val_transforms) #Why are we using val_transforms for Test Data?
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = config['batch_size'], shuffle = False,
                         drop_last = False, num_workers = 2)

In [ ]:
print("Number of classes: ", len(train_dataset.classes))
print("No. of train images: ", train_dataset.__len__())
print("Shape of image: ", train_dataset[0][0].shape)
print("Batch size: ", config['batch_size'])
print("Train batches: ", train_loader.__len__())
print("Val batches: ", val_loader.__len__())

Number of classes:  7000
No. of train images:  140000
Shape of image:  torch.Size([3, 224, 224])
Batch size:  128
Train batches:  1094
Val batches:  274


# Convnext-S

In [ ]:
class ConvNextBlock(torch.nn.Module):
  
  def __init__(self,channels):
    super().__init__()
    #ToDO
    self.channels=channels
    self.block=torch.nn.Sequential(
        torch.nn.Conv2d(self.channels,self.channels,kernel_size=7,stride=1,padding=3,groups=self.channels),
        torch.nn.BatchNorm2d(self.channels),
        torch.nn.Conv2d(self.channels,self.channels*4,kernel_size=1,stride=1),
        torch.nn.GELU(),
        torch.nn.Conv2d(self.channels*4,self.channels,kernel_size=1,stride=1)
        
        
    )
  def forward(self,x):
    res=x+self.block(x)
    return res

# cnext=ConvNextBlock(3).to(device)
# summary(cnext,(3,24,24))

In [ ]:
class ConvNext(torch.nn.Module):
  
  def __init__(self,in_channels,num_classes):
    super().__init__()
    self.stem=torch.nn.Sequential(
        torch.nn.Conv2d(in_channels,96,kernel_size=4,stride=4),
        torch.nn.BatchNorm2d(96)
    )

    self.backbone=torch.nn.Sequential(
        ConvNextBlock(96),
        ConvNextBlock(96),
        ConvNextBlock(96),

        torch.nn.BatchNorm2d(96),
        torch.nn.Conv2d(96,192,kernel_size=2,stride=2),

        ConvNextBlock(192),
        ConvNextBlock(192),
        ConvNextBlock(192),

        torch.nn.BatchNorm2d(192),
        torch.nn.Conv2d(192,384,kernel_size=2,stride=2),

        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),

        torch.nn.BatchNorm2d(384),
        torch.nn.Conv2d(384,768,kernel_size=2,stride=2),

        ConvNextBlock(768),
        ConvNextBlock(768),
        ConvNextBlock(768),

        torch.nn.AdaptiveAvgPool2d(1),
        torch.nn.BatchNorm2d(768),
        torch.nn.Flatten()
        #33 million
    )
    self.cls_layer=torch.nn.Linear(768,num_classes)
  
  def forward(self,x,return_feats=False):
    out=self.stem(x)
    out=self.backbone(out)
    if return_feats:
      return out
    return self.cls_layer(out)
model=ConvNext(3,7000).to(device)
summary(model,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 56, 56]           4,704
       BatchNorm2d-2           [-1, 96, 56, 56]             192
            Conv2d-3           [-1, 96, 56, 56]           4,800
       BatchNorm2d-4           [-1, 96, 56, 56]             192
            Conv2d-5          [-1, 384, 56, 56]          37,248
              GELU-6          [-1, 384, 56, 56]               0
            Conv2d-7           [-1, 96, 56, 56]          36,960
     ConvNextBlock-8           [-1, 96, 56, 56]               0
            Conv2d-9           [-1, 96, 56, 56]           4,800
      BatchNorm2d-10           [-1, 96, 56, 56]             192
           Conv2d-11          [-1, 384, 56, 56]          37,248
             GELU-12          [-1, 384, 56, 56]               0
           Conv2d-13           [-1, 96, 56, 56]          36,960
    ConvNextBlock-14           [-1, 96,

# Setup everything for training

In [ ]:
# PATH='/content/drive/MyDrive/Colab Notebooks/11485hw2model_convnext_1.pth'
# checkpoint = torch.load(PATH)
# model.load_state_dict(checkpoint['model_state_dict'])
# val_acc = checkpoint['val_acc']
# print(val_acc)

89.78387317518248


In [ ]:
#criterion = # TODO: What loss do you need for a multi class classification problem?
criterion= torch.nn.CrossEntropyLoss(label_smoothing=0.25)  #don't go beyond 0.25
#look at cosine similarity, etc. different types of losses 
#Train with cross entropy and then save the model, train further with different loss
#different ways to go about this.

optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-5) #increase weight decay after plateau
#optimizer = torch.optim.SGD(model.parameters(), lr=config['lr'], momentum=0.9, weight_decay=1e-5) #increase weight decay after plateau
# TODO: Implement a scheduler (Optional but Highly Recommended)
# You can try ReduceLRonPlateau, StepLR, MultistepLR, CosineAnnealing, etc.
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

#ReduceLRonPlateau,cosineAnnealing > MultiStepLR for ResNet 
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[9],gamma=0.5) 

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                               mode='max',  #max for maximizing validation accuracy 
                                                               factor=0.5,
                                                               patience=3,  #reduces lr faster if low
                                                               min_lr=1e-6,
                                                               verbose=True)
#scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100*len(train_loader), eta_min=0, last_epoch=- 1, verbose=False)
#100=epochs


#With Lr=0.1, get around 68% validation accuracyt with reduceLRonPlateau with augmentation. 

scaler = torch.cuda.amp.GradScaler() # Good news. We have FP16 (Mixed precision training) implemented for you
# It is useful only in the case of compatible GPUs such as T4/V100

In [ ]:
optimizer


SGD (
Parameter Group 0
    dampening: 0
    lr: 0.006
    momentum: 0.9
    nesterov: False
    weight_decay: 1e-05
)

# Let's train!

In [ ]:
def train(model, dataloader, optimizer, criterion):
    
    model.train()

    # Progress Bar 
    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, leave=False, position=0, desc='Train', ncols=5) 
    
    num_correct = 0
    total_loss = 0

    for i, (images, labels) in enumerate(dataloader):
        
        optimizer.zero_grad() # Zero gradients

        images, labels = images.to(device), labels.to(device)
        
        with torch.cuda.amp.autocast(): # This implements mixed precision. Thats it! 
            outputs = model(images)
            loss = criterion(outputs, labels)

        # Update no. of correct predictions & loss as we iterate
        num_correct += int((torch.argmax(outputs, axis=1) == labels).sum())
        L=float(loss.item())
        total_loss += L

        # tqdm lets you add some details so you can monitor training as you train.
        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * num_correct / (config['batch_size']*(i + 1))),
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            num_correct=num_correct,
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() 

        # TODO? Depending on your choice of scheduler,
        # You may want to call some schdulers inside the train function. What are these?
        #some scheudlers work on epoch;some on iteration. 
        #ex. cosine for iteration, reduceLR for epoch so call out in epoch loop. 
        batch_bar.update() # Update tqdm bar
    
    batch_bar.close() # You need this to close the tqdm bar

    acc = 100 * num_correct / (config['batch_size']* len(dataloader))
    total_loss = float(total_loss / len(dataloader))

    return acc, total_loss

In [ ]:
def validate(model, dataloader, criterion):
  
    model.eval()
    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, position=0, leave=False, desc='Val', ncols=5)

    num_correct = 0.0
    total_loss = 0.0

    for i, (images, labels) in enumerate(dataloader):
        
        # Move images to device
        images, labels = images.to(device), labels.to(device)
        
        # Get model outputs
        with torch.inference_mode():
            outputs = model(images)
            loss = criterion(outputs, labels)

        num_correct += int((torch.argmax(outputs, axis=1) == labels).sum())
        total_loss += float(loss.item())

        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * num_correct / (config['batch_size']*(i + 1))),
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            num_correct=num_correct)

        batch_bar.update()
        
    batch_bar.close()
    acc = 100 * num_correct / (config['batch_size']* len(dataloader))
    total_loss = float(total_loss / len(dataloader))
    return acc, total_loss

In [ ]:
gc.collect() # These commands help you when you face CUDA OOM error
torch.cuda.empty_cache()

# Wandb

In [ ]:
wandb.login(key="b60bfe3e01a4340c8db67c2772c12aaf0441572f") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Create your wandb run
run = wandb.init(
    name = "Oct24 with convnext-s continuation", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw2p2-ablations", ### Project should be created in your wandb account 
    config = config ### Wandb Config for your run
)

Problem at: <ipython-input-74-c29bd1aa21d6> 8 <module>


KeyboardInterrupt: ignored

In [ ]:
config['lr']

0.006

In [ ]:
optimizer.param_groups[0]['lr']

0.0015

# Experiments

In [ ]:
#best_valacc = checkpoint['val_acc']
best_valacc = 0

for epoch in range(config['epochs']):

    curr_lr = float(optimizer.param_groups[0]['lr'])

    train_acc, train_loss = train(model, train_loader, optimizer, criterion)
    
    print("\nEpoch {}/{}: \nTrain Acc {:.04f}%\t Train Loss {:.04f}\t Learning Rate {:.04f}".format(
        epoch + 1,
        config['epochs'],
        train_acc,
        train_loss,
        curr_lr))
    
    val_acc, val_loss = validate(model, val_loader, criterion)
    scheduler.step(val_acc)
    print("Val Acc {:.04f}%\t Val Loss {:.04f}".format(val_acc, val_loss))

    #wandb.log({"train_loss":train_loss, 'train_Acc': train_acc, 'validation_Acc':val_acc, 
               #'validation_loss': val_loss, "learning_Rate": curr_lr})
    
    # If you are using a scheduler in your train function within your iteration loop, you may want to log
    # your learning rate differently 

    # #Save model in drive location if val_acc is better than best recorded val_acc
    if val_acc >= best_valacc:
      #path = os.path.join(root, model_directory, 'checkpoint' + '.pth')
      print("Saving model")
      torch.save({'model_state_dict':model.state_dict(),
                  'optimizer_state_dict':optimizer.state_dict(),
                  'scheduler_state_dict':scheduler.state_dict(),
                  'val_acc': val_acc, 
                  'epoch': epoch}, '/content/drive/MyDrive/Colab Notebooks/11485hw2model_convnext_1.pth') 
      #'./drive/MyDrive/2022_Fall/11485hw2modelversion2.pth' is second attempt with extra data augmentations 
      #./drive/MyDrive/2022_Fall/11485hw2model.pth is first attempt that made it to 86
      best_valacc = val_acc
      #wandb.save('checkpoint.pth')
      # You may find it interesting to exlplore Wandb Artifcats to version your models
run.finish()

Train:   0%|          | 0/1094 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Epoch 1/100: 
Train Acc 98.4204%	 Train Loss 2.9842	 Learning Rate 0.0010


Val Acc 89.8209%	 Val Loss 3.7487
Saving model



Epoch 2/100: 
Train Acc 98.4768%	 Train Loss 2.9810	 Learning Rate 0.0010


Val Acc 89.7867%	 Val Loss 3.7487


Train:  38%|███▊      | 413/1094 [09:43<15:47,  1.39s/it, acc=98.4828%, loss=2.9829, lr=0.0010, num_correct=52188]

KeyboardInterrupt: ignored

# Classification Task: Testing

In [ ]:
def test(model,dataloader):

  model.eval()
  batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, position=0, leave=False, desc='Test')
  test_results = []
  
  for i, (images) in enumerate(dataloader):
      images = images.to(device)

      with torch.inference_mode():
        outputs = model(images)

      outputs = torch.argmax(outputs, axis=1).detach().cpu().numpy().tolist()
      test_results.extend(outputs)
      
      batch_bar.update()
      
  batch_bar.close()
  return test_results

In [ ]:
test_results = test(model, test_loader)

## Generate csv to submit to Kaggle

In [ ]:
with open("classification_early_submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(test_dataset)):
        f.write("{},{}\n".format(str(i).zfill(6) + ".jpg", test_results[i]))

# Verification Task: Validation

The verification task consists of the following generalized scenario:
- You are given X unknown identitites 
- You are given Y known identitites
- Your goal is to match X unknown identities to Y known identities.

We have given you a verification dataset, that consists of 1000 known identities, and 1000 unknown identities. The 1000 unknown identities are split into dev (200) and test (800). Your goal is to compare the unknown identities to the 1000 known identities and assign an identity to each image from the set of unknown identities. 

Your will use/finetune your model trained for classification to compare images between known and unknown identities using a similarity metric and assign labels to the unknown identities. 

This will judge your model's performance in terms of the quality of embeddings/features it generates on images/faces it has never seen during training for classification.

In [ ]:
known_regex = "/content/data/verification/known/*/*"
known_paths = [i.split('/')[-2] for i in sorted(glob.glob(known_regex))] 
# This obtains the list of known identities from the known folder

unknown_regex = "/content/data/verification/unknown_test/*" #Change the directory accordingly for the test set
#switch between
# We load the images from known and unknown folders
unknown_images = [Image.open(p) for p in tqdm(sorted(glob.glob(unknown_regex)))]
known_images = [Image.open(p) for p in tqdm(sorted(glob.glob(known_regex)))]

# Why do you need only ToTensor() here?
transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()])

unknown_images = torch.stack([transforms(x) for x in unknown_images])
known_images  = torch.stack([transforms(y) for y in known_images ])
#Print your shapes here to understand what we have done

# You can use other similarity metrics like Euclidean Distance if you wish
similarity_metric = torch.nn.CosineSimilarity(dim= 1, eps= 1e-6) 


100%|██████████| 800/800 [00:00<00:00, 3288.56it/s]

100%|██████████| 1000/1000 [00:00<00:00, 3687.34it/s]


In [ ]:
def eval_verification(unknown_images, known_images, model, similarity, batch_size= config['batch_size'], mode='val'): 

    unknown_feats, known_feats = [], []

    batch_bar = tqdm(total=len(unknown_images)//batch_size, dynamic_ncols=True, position=0, leave=False, desc=mode)
    model.eval()

    # We load the images as batches for memory optimization and avoiding CUDA OOM errors
    for i in range(0, unknown_images.shape[0], batch_size):
        unknown_batch = unknown_images[i:i+batch_size] # Slice a given portion upto batch_size
        
        with torch.no_grad():
            # unknown_feat = model(unknown_batch.float().to(device), return_feats=True) #Get features from model         
            unknown_feat = model(unknown_batch.float().to(device))
        unknown_feats.append(unknown_feat)
        batch_bar.update()
    
    batch_bar.close()
    
    batch_bar = tqdm(total=len(known_images)//batch_size, dynamic_ncols=True, position=0, leave=False, desc=mode)
    
    for i in range(0, known_images.shape[0], batch_size):
        known_batch = known_images[i:i+batch_size] 
        with torch.no_grad():
              #known_feat = model(known_batch.float().to(device), return_feats=True)
              known_feat = model(known_batch.float().to(device))
          
        known_feats.append(known_feat)
        batch_bar.update()

    batch_bar.close()

    # Concatenate all the batches
    unknown_feats = torch.cat(unknown_feats, dim=0)
    known_feats = torch.cat(known_feats, dim=0)

    similarity_values = torch.stack([similarity(unknown_feats, known_feature) for known_feature in known_feats])
    # Print the inner list comprehension in a separate cell - what is really happening?

    predictions = similarity_values.argmax(0).cpu().numpy() #Why are we doing an argmax here?

    # Map argmax indices to identity strings
    pred_id_strings = [known_paths[i] for i in predictions]
    
    if mode == 'val':
      true_ids = pd.read_csv('/content/data/verification/dev_identities.csv')['label'].tolist()
      accuracy = accuracy_score(pred_id_strings, true_ids)
      print("Verification Accuracy = {}".format(accuracy))
    
    return pred_id_strings

In [ ]:
pred_id_strings = eval_verification(unknown_images, known_images, model, similarity_metric, config['batch_size'], mode='test')

In [ ]:
with open("verification_early_submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(pred_id_strings)):
        f.write("{},{}\n".format(i, pred_id_strings[i]))

In [ ]:
!kaggle competitions submit -c 11-785-f22-hw2p2-classification -f classification_early_submission.csv -m "Message"

100% 541k/541k [00:00<00:00, 2.24MB/s]
Successfully submitted to Face Recognition

In [ ]:
!kaggle competitions submit -c 11-785-f22-hw2p2-verification -f verification_early_submission.csv -m "Message"

100% 9.28k/9.28k [00:00<00:00, 43.2kB/s]
Successfully submitted to Face Verification